In [260]:
import pandas as pd

df = pd.read_parquet('./data/')
df.drop('0_price', axis=1, inplace=True)
df.drop('1_price', axis=1, inplace=True)
df.drop('2_price', axis=1, inplace=True)
df.drop('3_price', axis=1, inplace=True)
df.drop('4_price', axis=1, inplace=True)

df.loc[df['wave_direction'] == 'min', 'wave_direction'] = 1
df.loc[df['wave_direction'] == 'max', 'wave_direction'] = -1
print(f'max spread: {df["stabilized_spread"].max()}')
df = df[df['stabilized_spread'] >= 0.2]
df = df[df['stabilized_spread'] < 8]
df

max spread: 23.59


,0_amount_mean,0_nr_trades,0_spread,1_amount_mean,1_nr_trades,1_spread,2_amount_mean,2_nr_trades,2_spread,3_amount_mean,...,stabilized_amount_mean,stabilized_at_ms,stabilized_nr_trades,stabilized_spread,wave_direction,0_price_delta,1_price_delta,2_price_delta,3_price_delta,4_price_delta
index,,,,,,,,,,,,,,,,,,,,,
0,0.0140,3.0,0.37,0.0125,4.0,0.37,0.0176,12.0,0.62,0.0491,...,0.0448,214,21.0,0.77,-1,0.40,0.40,0.15,0.10,0.00
0,0.0270,6.0,0.48,0.0463,9.0,0.52,0.0423,11.0,0.52,0.0462,...,0.0415,91,19.0,0.79,1,0.31,0.27,0.27,0.07,0.03
0,0.0012,3.0,0.44,0.0022,4.0,0.57,0.0019,5.0,0.57,0.0019,...,0.0046,221,8.0,0.78,1,0.34,0.21,0.21,0.20,0.00
0,0.0417,4.0,0.69,0.0221,10.0,0.79,0.0202,11.0,0.79,0.0573,...,0.0526,300,14.0,0.95,-1,0.26,0.16,0.16,0.16,0.08
0,0.0053,3.0,0.33,0.0569,4.0,0.39,0.0551,5.0,0.39,0.0527,...,0.0488,185,11.0,0.51,-1,0.18,0.12,0.12,0.02,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.0170,5.0,1.06,0.0144,6.0,1.06,0.0293,7.0,1.06,0.0268,...,0.0462,56,19.0,1.79,1,0.73,0.73,0.73,0.73,0.72
0,0.0221,12.0,1.64,0.0223,16.0,1.81,0.0198,19.0,1.81,0.0197,...,0.0187,122,22.0,1.84,1,0.03,0.03,0.03,0.03,0.00
0,0.0246,7.0,1.32,0.0282,8.0,1.48,0.0445,9.0,1.48,0.0405,...,0.0347,125,12.0,1.60,-1,0.28,0.12,0.12,0.12,0.12


In [261]:
df = df.reset_index(drop=True)
df.iloc[5:8]

,0_amount_mean,0_nr_trades,0_spread,1_amount_mean,1_nr_trades,1_spread,2_amount_mean,2_nr_trades,2_spread,3_amount_mean,...,stabilized_amount_mean,stabilized_at_ms,stabilized_nr_trades,stabilized_spread,wave_direction,0_price_delta,1_price_delta,2_price_delta,3_price_delta,4_price_delta
5,0.0082,3.0,0.02,0.0088,4.0,0.65,0.0070,5.0,0.65,0.0064,...,0.0051,395,8.0,1.11,1,0.46,0.46,0.46,0.02,0.00
6,0.0288,5.0,0.89,0.0297,13.0,1.42,0.0279,14.0,1.44,0.0316,...,0.0274,141,20.0,1.49,-1,0.60,0.07,0.05,0.00,0.00
7,0.0117,4.0,0.23,0.0105,5.0,0.40,0.0104,6.0,0.42,0.0105,...,0.0085,242,10.0,0.55,1,0.32,0.15,0.13,0.13,0.13


In [262]:
df['last_price_delta_since_stabilized'].max()

36.58

In [263]:
from sklearn.model_selection import train_test_split

y = df['last_price_delta_since_stabilized']
X = df.drop('last_price_delta_since_stabilized', axis=1)
X = X.reset_index(drop=True)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)
X

,0_amount_mean,0_nr_trades,0_spread,1_amount_mean,1_nr_trades,1_spread,2_amount_mean,2_nr_trades,2_spread,3_amount_mean,...,stabilized_amount_mean,stabilized_at_ms,stabilized_nr_trades,stabilized_spread,wave_direction,0_price_delta,1_price_delta,2_price_delta,3_price_delta,4_price_delta
0,0.0140,3.0,0.37,0.0125,4.0,0.37,0.0176,12.0,0.62,0.0491,...,0.0448,214,21.0,0.77,-1,0.40,0.40,0.15,0.10,0.00
1,0.0270,6.0,0.48,0.0463,9.0,0.52,0.0423,11.0,0.52,0.0462,...,0.0415,91,19.0,0.79,1,0.31,0.27,0.27,0.07,0.03
2,0.0012,3.0,0.44,0.0022,4.0,0.57,0.0019,5.0,0.57,0.0019,...,0.0046,221,8.0,0.78,1,0.34,0.21,0.21,0.20,0.00
3,0.0417,4.0,0.69,0.0221,10.0,0.79,0.0202,11.0,0.79,0.0573,...,0.0526,300,14.0,0.95,-1,0.26,0.16,0.16,0.16,0.08
4,0.0053,3.0,0.33,0.0569,4.0,0.39,0.0551,5.0,0.39,0.0527,...,0.0488,185,11.0,0.51,-1,0.18,0.12,0.12,0.02,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1956,0.0170,5.0,1.06,0.0144,6.0,1.06,0.0293,7.0,1.06,0.0268,...,0.0462,56,19.0,1.79,1,0.73,0.73,0.73,0.73,0.72
1957,0.0221,12.0,1.64,0.0223,16.0,1.81,0.0198,19.0,1.81,0.0197,...,0.0187,122,22.0,1.84,1,0.03,0.03,0.03,0.03,0.00
1958,0.0246,7.0,1.32,0.0282,8.0,1.48,0.0445,9.0,1.48,0.0405,...,0.0347,125,12.0,1.60,-1,0.28,0.12,0.12,0.12,0.12
1959,0.0021,3.0,0.17,0.0026,4.0,0.17,0.0029,5.0,0.17,0.0026,...,0.0036,89,8.0,0.94,1,0.77,0.77,0.77,0.77,0.61


In [269]:
from datetime import datetime
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

model = RandomForestRegressor(n_estimators=100, random_state=0, max_depth=5)
model.fit(X_train, y_train)

preds = model.predict(X_valid)
preds
y_valid.reset_index(drop=True)

mae = round(mean_absolute_error(y_valid, preds), 5)
print(f'mean absolute error: {mae}')

mean absolute error: 1.83639


In [189]:

rec = len(X_train)
filename = f'models/{datetime.utcnow().replace(microsecond=0).isoformat()}-rec-{rec}-mae-{mae}.sav'
pickle.dump(model, open(filename, 'wb'))
print(f'saved model to {filename}')

mean absolute error: 0.69432
saved model to models/2023-01-19T18:39:36-rec-7323-mae-0.69432.sav


In [136]:
model = pickle.load(open(filename, 'rb'))
wave_frame = [{'stabilized_at_ms': 14, 'stabilized_nr_trades': 13, 'stabilized_amount_mean': 0.0344, 'stabilized_spread': 0.19, 'wave_direction': 'min', 'last_price_delta_since_stabilized': 1.26}]
df = pd.DataFrame(wave_frame)
df.loc[df['wave_direction'] == 'min', 'wave_direction'] = 1
df.loc[df['wave_direction'] == 'max'] = -1
df.drop('last_price_delta_since_stabilized', axis=1, inplace=True)
result = model.predict(df)
print(result)


[0.81305]


### pipeline approach

In [100]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.model_selection import train_test_split


class FeatureEngineer(BaseEstimator, TransformerMixin):
    def transform(self, X, y=None):
        df = X.copy()
        df.loc[df['wave_direction'] == 'min', 'wave_direction'] = 1
        df.loc[df['wave_direction'] == 'max'] = -1
        df.drop('last_price_delta_since_stabilized', axis=1, inplace=True)
        return df


pipeline = Pipeline(steps=[
    ('preprocessor', FeatureEngineer()),
    ('model', RandomForestRegressor(n_estimators=50, random_state=0))
])

df = pd.read_parquet('./data')
y = df['last_price_delta_since_stabilized']
X = df.drop('last_price_delta_since_stabilized', axis=1)
X = X.reset_index(drop=True)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.2, test_size=0.2, random_state=0)
pipeline.fit(X_train, y_train)



AttributeError: 'FeatureEngineer' object has no attribute 'fit'